This is an implementation of depolying CNN gensture reconition model to edge device (e.g. Sony Spresense)

Step Overview:
1. Conver Pytorch Model to Onnx Model
2. Conver Onnx Model to Keras Model
3. Conver Keras Model to quantization aware model
4. Retraining quantization aware model
5. Convert it to Tensorflow Lite Model
6. Convert the model to hex header file


## Create Representive Data

In [ ]:
import numpy as np
import scipy.io
import torch
import json
import os
config = json.load(open('config.json', 'r'))
# Load input data from a MATLAB file
for i in range(1,9):
    input_data = scipy.io.loadmat(os.path.join(config["data_path"],f"001-00{i}-005.mat"))['Data']

    # Convert the input data to a PyTorch tensor
    input_data = torch.asarray(input_data)
    if input_data.shape[1] == 193:
        input_data = input_data[:,:-1]
    # Reshape the input data to match the expected shape of the model
    input_data = input_data.reshape(-1, 1, 8, 24)

    input_data = input_data.cpu().detach().numpy()

    np.save(f"representive_data_{i-1}",input_data)

# Conver Pytorch Model to Onnx Model

In [ ]:
import torch
import torchvision
from models.mobilenetv1 import MobilenetV1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dummy_input = torch.randn(1, 1, 8, 24, device=device)
model = MobilenetV1(ch_in=1, n_classes=8, Global_ratio=0.3).to(device)
model.load_state_dict(torch.load(
    "pretrain_model/0.3MobilenetV1_Param@303.31 k_MAC@524.1 KMac_Acc@95.761.pt"))
model.eval()

# print(model)
torch.onnx.export(model, dummy_input,
                  "pretrain_model/onnx_model/MobilenetV1.onnx", verbose=True)


# Conver Onnx Model to Keras Model

In [ ]:
# Install library
%cd onnx2keras
!pip install -e .
%cd ..

In [ ]:
import tensorflow as tf
import onnx

onnx_model = onnx.load("pretrain_model/onnx_model/MobilenetV1.onnx")
from onnx2keras import onnx_to_keras
model = onnx_to_keras(onnx_model, ['input.1'],name_policy='renumerate',verbose=False,change_ordering=True)
model.summary()

## Quantization aware training

In [ ]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

from utils.ICE_lab_data_preprocessing import ICE_lab_data_preprocessing as utils

data,label,num_classes = utils().extra_data("data/Training_Trimmed")
from sklearn.model_selection import train_test_split

training_data, testing_data, training_label, testing_label = train_test_split(data, label, test_size=0.33, random_state=42)
train_data = tf.data.Dataset.from_tensor_slices((training_data, training_label))
test_data = tf.data.Dataset.from_tensor_slices((testing_data, testing_label))

training_data = training_data.reshape(-1,8,24,1)
testing_data = testing_data.reshape(-1,8,24,1)
q_aware_model.fit(training_data,training_label,
                  batch_size=1000, epochs=2)
_, q_aware_model_accuracy = q_aware_model.evaluate(
   testing_data, testing_label, batch_size=1000,verbose=True)
print('Quant test accuracy:', q_aware_model_accuracy)




In [ ]:
q_aware_model.save("pretrain_model/q_ware_model")

# Convert it to Tensorflow Lite Model

In [ ]:
import numpy as np
import os
def representative_dataset():
    data = np.load("representive_data_0.npy")
    for i in range(1):
        temp_data = data[i]
        temp_data = temp_data.reshape(1,8,24,1)
        yield [temp_data.astype(np.float32)]

import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("pretrain_model/q_ware_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()
tflite_model_size = len(tflite_model) / 1024
print('Quantized model size = %dKBs.' % tflite_model_size)
# Save the model
with open("pretrain_model/tf_lite_model/mobilenetv1.tflite", 'wb') as f:
    f.write(tflite_model)

# Simulate Model Accuracy

In [ ]:
import numpy as np
import tensorflow as tf
import os

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="pretrain_model/tf_lite_model/mobilenetv1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.int8)
for j in range(8):
    ori_input_data = np.load(f"representive_data_{j}.npy")
    ori_input_data = ori_input_data.astype(np.float32)
    # ori_input_data = ori_input_data.reshape(-1,8,24,1)
    correct = 0
    print("Total Sample Size:",ori_input_data.shape[0])
    for i in range(ori_input_data.shape[0]):
        input_data = np.expand_dims(ori_input_data[i], 0)
        input_data = input_data.reshape(-1,8,24,1)
        interpreter.set_tensor(input_details[0]['index'], input_data)

        interpreter.invoke()

        # get_tensor() returns a copy of the tensor data
        # use tensor() in order to get a pointer to the tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        if np.argmax(output_data) == j:
            correct += 1
    print("Prediction Correct Size:",correct) #Total:30720
    print("Accuracy",round(correct/int(ori_input_data.shape[0]),2))

# Convert TFlite file to hex header file

In [ ]:
import binascii
def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

tflite_binary = open('pretrain_model/tf_lite_model/mobilenetv1.tflite', 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
header_file = "const unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
with open("pretrain_model/tf_lite_model/model.h", "w") as f:
    f.write(header_file)